# Preleminaries

- Possible Questions to answer: Most popular brawler/map/game mode, etc.
- Train a model on e.g. solo show down/duo show down/gem grep, one model per map, which brawler/brawlers are most likely to win according to battle logs
- some more questions... which star power/gadeget is most common or the like

In [ ]:
import os, json
import pandas as pd # for data stuff
import numpy as np # for importing data i think, not sure anymore
import matplotlib.pyplot as plt #for plotting

# the modules below are used in "Analys Models" section
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from pandas.io.json import json_normalize


Install Brawl Stars API with pip
 - https://brawlstats.readthedocs.io/en/latest/index.html

In [ ]:
!pip install brawlstats

# Data Collection

- collect Data from brawl stars API

Import brawlstats 

In [ ]:
import brawlstats
import time

Establish connection with token to Brawl Stars API
- create account at: https://developer.brawlstars.com/#/
- get ip with: https://www.whatismyip.org/
- create API Key on https://developer.brawlstars.com/#/ for your account and your API Key
- save API key locally in a file at: ../token_brawlstats.txt (so it is not part of this git repository)

In [ ]:
path = os.getcwd()
token_file =  open(os.path.join(path, "token_brawlstats.txt"), 'r')
token = token_file.read()
token_file.close()

client = brawlstats.Client(token)

- retrieve the top 200 players
- we assume they all have their brawlers on max level including gadgets (maybe plot this?), so there is no difference in the strength of the brawlers itself
- we assume their skill level is the same, this enables results in battlelogs of players with similar skill level

In [ ]:
tags_top_200_players = [player["tag"] for player in client.get_rankings(ranking='players', limit=200)]
top_200_players = [client.get_player(tag) for tag in tags_top_200_players]

- fetch the all available battlelogs of the 200 top players
- this data is always up do date
- we store it in a json file

In [ ]:
# initialize brawler levels count to 0
# brawlers can be from level 1 to 10
brawler_levels = {}
for i in range(1, 11):
    brawler_levels[i] = 0
    
# loop over the top 200 players
for player in top_200_players:
    
    # loop over the brawlers of the current player
    for brawler in player.brawlers:
        # increment
        brawler_levels[brawler["power"]] += 1

for level in brawler_levels:
    print("{} : {}".format(level ,brawler_levels[level]))

In [ ]:
battlelogs = {}
index = 0

with open("battlelogs.json", "w") as battlelog_file:
    for player in top_200_players:
        player_battlelogs = client.get_battle_logs(player.tag)
        for battlelog in player_battlelogs:
            battlelogs[index] = {}
            battlelogs[index]["player_tag"] = player.tag
            battlelogs[index]["battlelog"] = battlelog
            index += 1
    json.dump(battlelogs, battlelog_file)
        
        

# Data Processing
- maybe just plot the data, see if there is somethin weird

In [ ]:
    with open("battlelogs.json",'r') as lst:
        battlelogs = json.load(lst)

        unique_dicts = []
        used_battle_times = []
        for key in battlelogs.keys():
            current_battle_time = battlelogs[key]['battlelog']["battle_time"]
            if not current_battle_time in used_battle_times:
                used_battle_times.append(current_battle_time)
                unique_dicts.append(battlelogs[key])
        print(len(unique_dicts))
        
        ###print(unique_dicts[1])
        
        print(unique_dicts[1]["battlelog"]["battle_time"])
 
      

# Data Cleaning

- maybe write down what we figured out and how we got rid of it

In [ ]:
    with open("battlelogs.json",'r') as lst:
        b = json.load(lst)
        print(type(b))
        print(b)
       ## for key in b.keys():
            

# Data Cleaning

- maybe write down what we figured out and how we got rid of it

In [ ]:
print("Is e dreckig, musse putze")

# Preparation for Model Building

- do some stratification perhaps

In [ ]:
print("stratify data for training purposes")

# Train Model

In [ ]:
print("run code which uses train data to train the model")

# Test Model

In [ ]:
print("run code to test the model")

# Analyse Model

- probably some plots will be excpected

In [ ]:



# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Run classifier, using a model that is too regularized (C too low) to see
# the impact on the results
classifier = svm.SVC(kernel='linear', C=0.01).fit(X_train, y_train)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(classifier, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

# Visualize results

In [ ]:
print("some very nice plots like the ones above")

# Discussion or whatever

- write down our insights or maybe do some further coding in the cell below

In [ ]:
print("finally, done with this")